In [1]:
# Basic data analysis and data IO
import numpy as np
import pandas as pd
import pickle

# Network preprocssing 
import scipy.sparse as sp
import torch

# GNN 
from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T 
from torch_geometric.utils import negative_sampling

# Evaluation 
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score

# Clustering
from sklearn.cluster import KMeans
from collections import Counter

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import chart_studio
from chart_studio.plotly import plot

# ETC
from tqdm import tqdm
import warnings 

warnings.filterwarnings(action='ignore')
chart_studio.tools.set_credentials_file(username='Injokim', api_key='ZAsiIjcDIzXdYHa8CXbw')

# 1. Construct Data loader 

In [2]:
node_feature = 'basic'

In [3]:
"""
- Select feature information to be input to the GNN algorithm
- Mainly divided into two types: None and basic
     - None: contains no information, identity matrix as input, total length 39
     - basic : readme information 
"""

node_data = pd.read_csv('data/network/repo_topic/textgcn_network.csv', index_col=0).columns
n_repo = 3367
n_topic = 6456
repo_node, topic_node = node_data[n_topic:], node_data[:n_topic]

def feature_selector(network, feature=None) :
    if feature == 'basic' :
        
        # call document embedding vector
        with open('data/readme/2048_readme_vector.pickle', 'rb') as f :
            doc2vec = pickle.load(f)
        repo_feature = torch.tensor(doc2vec).to(torch.float)
        topic_feature = torch.zeros((n_topic, repo_feature.size(1))).to(torch.float)
        x = torch.cat((topic_feature, repo_feature), 0)
        
        # not use in this project 
        '''
        data = pd.read_csv('data/data/filtered_data.csv')

        # call social metric 
        need_feature = ['contributors_count', 'stargazers_count', 'forks_count', 'watchers_count']
        feature_data = data[need_feature]

        x = torch.tensor(feature_data.values).to(torch.float)
        x = torch.cat((x, doc2vec), 1)
        '''
        

    # add all genereated feature vectors
    elif feature == None :
        x = torch.eye(network.shape[0]).to(torch.float)
        
    return x

In [4]:
class MyData(InMemoryDataset) :
    def __init__(self, transform) :
        super(MyData, self).__init__()

        network = pd.read_csv('data/network/repo_topic/textgcn_network.csv', index_col=0)
        self.node_name = network.columns
        network = network.values

        adj = sp.csr_matrix(network).tocoo()
        row = torch.from_numpy(adj.row).to(torch.float)
        col = torch.from_numpy(adj.col).to(torch.float)

        edge_index = torch.stack([row, col], dim=0).to(torch.long)
        x = feature_selector(network, feature=node_feature)
        edge_weight = torch.tensor(adj.data).to(torch.long)

        data = Data(x=x, edge_index=edge_index, edge_attr=edge_weight)
        if transform : 
            data = transform(data)

        self.data, self.slices = self.collate([data])

- Slices contain shape information of data, but there is no need to use it.
- All information is included in the data, so you can just use it

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


transform_train = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=False,
                      add_negative_train_samples=True),
])

transform_embedding = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
])

# train splited data 
data_load = MyData(transform=transform_train)
train_data, val_data, test_data = data_load[0]

# embedding full data 
full_data = MyData(transform=transform_embedding)
data = full_data.data

- As a result of loading data, 4 basic information is included
    - x : original network, sparse network type
    - edge_index : A variant of the sparse network, including only pairs of indexes in which values ​​exist
    - edge_attribute : edge weight
    - train, val, test mask: indicates the purpose of each edge <br></br>

- Train, val, test split edge index, edge weight information is also included

---

# 2. Model construction

In [6]:
class GCNmodel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return prob_adj

---

# 3. GCN based link prediction

In [7]:
device = 'cpu'
hidden_size = 128
embedding_size = 64
epochs = 200
iteration = 1000

model = GCNmodel(data_load.num_features, hidden_size, embedding_size).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

In [8]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    out_binary = np.array([1 if ele >=0.95 else 0 for ele in out.cpu().numpy()])
    label = data.edge_label.cpu().numpy()
    return roc_auc_score(label, out.cpu().numpy()), f1_score(label, out_binary), precision_score(label, out_binary), recall_score(label, out_binary)

In [9]:
def training_visualization(auc, f1, precision, recall) :
    x_range = np.array(range(1, epochs+1))
    
    fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=('validation', 'test')
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=auc[0], mode='lines', name='val_auc'),
        row=1, col=1
    ) 
    
    fig.add_trace(
        go.Scatter(x=x_range, y=f1[0], mode='lines', name='val_f1'),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=precision[0], mode='lines', name='val_precision'),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=recall[0], mode='lines', name='val_recall'),
        row=1, col=1
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=auc[1], mode='lines', name='test_auc'),
        row=1, col=2
    ) 
    
    fig.add_trace(
        go.Scatter(x=x_range, y=f1[1], mode='lines', name='test_f1'),
        row=1, col=2
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=precision[1], mode='lines', name='test_precision'),
        row=1, col=2
    )

    fig.add_trace(
        go.Scatter(x=x_range, y=recall[1], mode='lines', name='test_recall'),
        row=1, col=2
    )

    fig.update_layout(height=500, width=900,
                  title_text="link prediction loss function graphs")
    fig.show()
    plot(fig, filename = 'training_graph', auto_open=True)


In [10]:
best_val_auc = final_test_auc = 0
final_test_f1 = final_test_precision = final_test_recall = 0

# visualize training process 
# add all validation and test metrics into list to draw graph 
val_auc_list, val_f1_list, val_precision_list, val_recall_list = [],[],[],[]
test_auc_list, test_f1_list, test_precision_list, test_recall_list = [],[],[],[]

# training parts
for epoch in range(1, epochs+1):
    loss = train()
    val_auc, val_f1, val_precision, val_recall = test(val_data)
    test_auc, test_f1, test_precision, test_recall = test(test_data)

    # for visualization 
    val_auc_list.append(val_auc); val_f1_list.append(val_f1); val_precision_list.append(val_precision); val_recall_list.append(val_recall)
    test_auc_list.append(test_auc); test_f1_list.append(test_f1); test_precision_list.append(test_precision); test_recall_list.append(test_recall)
    
    # find best metrics when training 
    if val_auc > best_val_auc:
        best_val = val_auc
        final_test_auc = test_auc
        final_test_f1 = test_f1
        final_test_precision = test_precision
        final_test_recall = test_recall 

    if epoch % 10 == 0 :
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
            f'Test: {test_auc:.4f}')

# for visualization 
auc_list = [val_auc_list, test_auc_list]
f1_list = [val_f1_list, test_f1_list]
precision_list = [val_precision_list, test_precision_list]
recall_list = [val_recall_list, test_recall_list]

print('\n')
print('Final Result')
print(f'Final Test AUC: {final_test_auc:.4f}')
print(f'Final Test F1 score: {final_test_f1:.4f}')
print(f'Final Test Precision: {final_test_precision:.4f}')
print(f'Final Test Recall: {final_test_recall:.4f}')


# conduct link prediction in currnet network 
# output form is adjacency matrix with logits 
z = model.encode(data.x, data.edge_index)
final_hetero = model.decode_all(z)

Epoch: 010, Loss: 0.7135, Val: 0.8167, Test: 0.8239
Epoch: 020, Loss: 0.6996, Val: 0.8528, Test: 0.8593
Epoch: 030, Loss: 0.6827, Val: 0.8402, Test: 0.8469
Epoch: 040, Loss: 0.6695, Val: 0.8598, Test: 0.8658
Epoch: 050, Loss: 0.6654, Val: 0.8262, Test: 0.8315
Epoch: 060, Loss: 0.6630, Val: 0.8327, Test: 0.8377
Epoch: 070, Loss: 0.6602, Val: 0.8509, Test: 0.8547
Epoch: 080, Loss: 0.6578, Val: 0.8545, Test: 0.8569
Epoch: 090, Loss: 0.6561, Val: 0.8577, Test: 0.8605
Epoch: 100, Loss: 0.6546, Val: 0.8536, Test: 0.8571
Epoch: 110, Loss: 0.6534, Val: 0.8518, Test: 0.8556
Epoch: 120, Loss: 0.6530, Val: 0.8529, Test: 0.8571
Epoch: 130, Loss: 0.6525, Val: 0.8503, Test: 0.8547
Epoch: 140, Loss: 0.6524, Val: 0.8578, Test: 0.8615
Epoch: 150, Loss: 0.6507, Val: 0.8455, Test: 0.8491
Epoch: 160, Loss: 0.6495, Val: 0.8629, Test: 0.8647
Epoch: 170, Loss: 0.6484, Val: 0.8656, Test: 0.8656
Epoch: 180, Loss: 0.6465, Val: 0.8630, Test: 0.8608
Epoch: 190, Loss: 0.6448, Val: 0.8644, Test: 0.8629
Epoch: 200, 

In [11]:
# visualization traing process 
training_visualization(auc_list, f1_list, precision_list, recall_list)

In [15]:
# Extract repo-topic heterogeneous network
# Conduct dichotomize
pred_repo_topic = final_hetero[:n_topic, n_topic:].sigmoid()
pred_repo_topic = torch.where(pred_repo_topic>0.8, 1, 0)

# 4. Predicted Quasi-topic network

In [16]:
# Create new quasi network
final_word = pred_repo_topic @ pred_repo_topic.T
final_word = pd.DataFrame(final_word.detach().numpy(), columns=topic_node, index=topic_node)

# remove diagonal term 
for node in final_word.columns :
    final_word.loc[node, node] = 0

In [17]:
final_word.to_csv('data/network/predicted_network/6_pred_word_network.csv')

In [ ]:
# edge normalization using association strength
node_diag = {node : final_word.loc[node, node] for node in final_word.columns}

norm_final_word = final_word.copy()
for node1 in tqdm(final_word.columns) :
    for node2 in final_word.columns :
        if node1 == node2 :
            norm_final_word.loc[node1, node2] = 0
        norm_final_word.loc[node1, node2] = norm_final_word.loc[node1, node2]/(node_diag[node1]*node_diag[node2])

In [ ]:
norm_final_word.to_csv('data/network/predicted_network/1_norm_final_network.csv')

# 5. Predicted network analysis 

## 5-1. Network filtering

In [ ]:
import networkx as nx
G = nx.from_pandas_adjacency(norm_final_word)

##  5-2. Network clustering

In [ ]:
embedding_vector = z.detach().numpy()

# 클러스터링 
kmeans = KMeans(n_clusters=20, random_state=11).fit(embedding_vector)
clusters = kmeans.labels_

# 노드에 클러스터 할당 
repo_cluster = {row.full_name : clusters[row_idx] for row_idx, row in node_data.iterrows()}
repo_cluster = pd.DataFrame.from_dict(repo_cluster, orient='index')

# 데이터프레임 정리
repo_cluster.reset_index(inplace=True)
repo_cluster.columns = ['Id', 'cluster']

In [ ]:
repo_cluster.to_csv('data/network/cluster.csv', index=False)

## 5-3. Check data distribution and topic distribution 

In [ ]:
repo_cluster = pd.read_csv('data/network/cluster.csv')

# 클러스터 분포 확인 
cluster_freq = pd.DataFrame.from_dict(Counter(repo_cluster.cluster), orient='index').reset_index()
cluster_freq.columns = ['cluster', 'freq']

In [ ]:
fig = px.bar(cluster_freq, x='cluster', y='freq')
plot(fig, filename='gcn_kmeans', auto_open=True)

In [ ]:
# 클러스터별 리드미 분류 
for cluster in np.unique(repo_cluster.cluster) :
    list(repo_cluster[repo_cluster.cluster==cluster].Id)
